In [1]:
import pandas as pd
import math
import numpy as np

import heapq
import numba as nb
from scipy.stats.mstats import gmean, hmean
from state import*

# data_full = pd.read_csv('data_test.csv')
# data_full = data_full.sort_values(by=['TIME', 'PROFIT'], ascending=[False, False], ignore_index=True)
# all_fomula = np.array(pd.read_csv('congthuc.csv')['fomula'])

data_full

c:\Users\Admin\Desktop\SYSTEM\RL_INVEST\state.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[1] = 1
c:\Users\Admin\Desktop\SYSTEM\RL_INVEST\state.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[2] = 'NOT_INVEST'
c:\Users\Admin\Desktop\SYSTEM\RL_INVEST\state.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s[4:] = np.average(data_full.loc[id: , data_full.columns[4:]], axis=0)
c:\Users\Admin\Desktop\SYSTEM\RL_INVEST\st

,TIME,PROFIT,SYMBOL,EXCHANGE,MARKET_CAP,Basic earnings per share,Other expenses,Financial expenses,VolumeARG,Financial income,...,Cash and cash equivalents,Short-term trade payables,Taxes and other obligations to the State Budget,Gross profit,Other short-term payables,In which: Loan interest expenses,Sales deductions,Short-term accrued expenses,LIABILITIES,Tangible fixed assets Accumulated depreciation
0,90,1.000000,NOT_INVEST,HOSE,8.873960e+12,231283.594397,8.469288e+09,5.072332e+10,8.857091e+05,6.373902e+10,...,6.778475e+11,5.893811e+11,1.097540e+11,3.105095e+11,4.307930e+11,3.586968e+10,1.243129e+10,3.037054e+11,4.984089e+12,-1.218414e+12
1,90,0.000000,AAA,HOSE,4.893114e+12,0.000000,3.926632e+09,5.918750e+10,5.156710e+06,6.215189e+10,...,1.018940e+12,9.794080e+11,2.943622e+10,3.430300e+11,2.816613e+11,3.589115e+10,3.873387e+09,1.870343e+10,5.338777e+12,-1.579764e+12
2,90,0.000000,AAT,HOSE,8.485598e+11,1384.000000,1.576288e+08,5.314787e+09,1.027330e+06,9.241194e+07,...,3.328005e+10,1.410469e+10,2.253305e+10,1.191474e+11,2.649353e+06,5.314787e+09,0.000000e+00,0.000000e+00,3.428906e+11,-1.813202e+11
3,90,0.000000,ABS,HOSE,9.451310e+11,0.000000,6.931579e+08,1.141712e+10,7.377700e+05,8.641370e+07,...,2.160073e+10,1.738495e+11,2.044294e+10,2.653042e+10,1.491120e+09,0.000000e+00,0.000000e+00,2.410694e+09,7.281878e+11,-7.332759e+09
4,90,0.000000,ACC,HOSE,1.911000e+12,258.000000,9.234700e+04,5.818817e+09,3.242300e+05,2.771447e+10,...,7.737720e+11,8.828534e+10,8.608350e+09,1.338862e+10,8.726839e+10,5.818817e+09,0.000000e+00,1.173808e+10,8.545696e+11,-1.158529e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7119,29,0.829231,GIL,HOSE,2.957500e+11,0.000000,1.178573e+07,4.434805e+08,2.435300e+04,1.918037e+08,...,5.592835e+09,2.042547e+10,1.471138e+08,1.327977e+10,9.091083e+08,2.843084e+08,0.000000e+00,2.477158e+08,4.939735e+10,-1.132837e+10
7120,29,0.792308,VSH,HOSE,8.125000e+12,0.000000,0.000000e+00,4.036377e+09,8.040500e+04,1.217429e+10,...,1.117020e+11,7.003186e+09,4.384513e+09,5.236953e+10,3.844896e+08,3.997361e+09,0.000000e+00,3.763020e+09,5.471286e+11,-1.213501e+12
7121,29,0.741068,FPT,HOSE,2.961458e+13,2550.000000,6.714888e+09,1.364417e+10,1.016800e+05,1.010007e+10,...,5.111802e+11,4.444050e+11,1.211761e+11,4.029315e+11,7.813338e+10,9.609989e+09,0.000000e+00,2.484622e+10,1.477624e+12,-2.382139e+11
7122,29,0.724551,GMD,HOSE,7.643971e+12,0.000000,1.093278e+07,9.889580e+09,1.322200e+05,2.829137e+10,...,1.335678e+12,0.000000e+00,0.000000e+00,5.367310e+10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.744527e+11,-2.710334e+11


In [2]:
len(index_test)

63

In [3]:
result, per_file = normal_main(player_random1, 1,[0])
result, per_file

<string>:1: RuntimeWarning: invalid value encountered in true_divide


(array([0., 1.]), [0])

In [4]:
result, per_file = normal_main(player_random1, 1000,[0])
result, per_file

<string>:1: RuntimeWarning: invalid value encountered in true_divide
<string>:1: RuntimeWarning: overflow encountered in multiply


(array([482., 518.]), [0])

In [2]:
result, per_file = normal_main(player_random1,  1,[0])
result, per_file

<string>:1: RuntimeWarning: invalid value encountered in true_divide


check getin4 1240 1
check getin4 1240 1
['(-(C*J*U*O*U*Y*L)/(C*J*U*O*U*Y*L)-(P*Q*J*P*I*E)/(P*Q*J*P*I*E)-(Q*C*O*S*B*I*O*D*K*Y*H)/(Q*C*O*S*B*I*O*D*K*Y*H)+(B*U*D*G*F*D*I*M*J*E*Z*O*W*K*K*S*C*Q*F*G)/(B*U*D*G*F*D*I*M*J*E*Z*O*W*K*K*S*C*Q*F*G))/A/A/A/A', '(-(E*U*O*S*L*G*L*W*Z*J*C*Q*P)/(E*U*O*S*L*G*L*W*Z*J*C*Q*P)-(D*B*B*U*N*T*S*Y*K*V*I)/(D*B*B*U*N*T*S*Y*K*V*I)+(O*O*U*I*C*F*S*F*I*J*Z)/(O*O*U*I*C*F*S*F*I*J*Z)-(T*C*I*I*R*I*F*H*C)+(F*B*J*X*L*Q*H*G*P*I*B*Z*I*T)/(F*B*J*X*L*Q*H*G*P*I*B*Z*I*T)-(R*V*K*O*R*O*G*O*M*M*E*T*X*D*G)/(R*V*K*O*R*O*G*O*M*M*E*T*X*D*G)+(N*S*K*W*K*N*V*D*V*W*F*I*O*I*N)/(N*S*K*W*K*N*V*D*V*W*F*I*O*I*N)-(W*R*L*H*K*L*M*X*P*S*K*H*E*C*X)/(W*R*L*H*K*L*M*X*P*S*K*H*E*C*X))/A/A/A/A/A/A/A/A/A']
check check [ 2  2  3  3  3  3  2  2  3  2  6  6 10 10  7  8  8  6  5  5  9  6  7  8
  9  9  9 10  9 11 12 10  9  8 12 12 14 13 12 15 13 16 14 18 16 15 14 14
 14 12 15  9 12 11 11 15 17 16 17 21 19 19] [ 2  2  4  2  4  3  2  1  4  2  7  4 16 10  9 10  7  6  2  3  9  9  9 11
 14 12 16 10 14 16 19 15 16 16

(array([0., 1.]), [0])

In [3]:
LIST_RANK_NOT_INVEST_CT1


array([ 2,  2,  3,  3,  3,  3,  2,  2,  3,  2,  6,  6, 10, 10,  7,  8,  8,
        6,  5,  5,  9,  6,  7,  8,  9,  9,  9, 10,  9, 11, 12, 10,  9,  8,
       12, 12, 14, 13, 12, 15, 13, 16, 14, 18, 16, 15, 14, 14, 14, 12, 15,
        9, 12, 11, 11, 15, 17, 16, 17, 21, 19, 19], dtype=int64)

In [4]:
LIST_RANK_NOT_INVEST_CT2

array([ 2,  2,  4,  2,  4,  3,  2,  1,  4,  2,  7,  4, 16, 10,  9, 10,  7,
        6,  2,  3,  9,  9,  9, 11, 14, 12, 16, 10, 14, 16, 19, 15, 16, 16,
       20, 16, 23, 15, 17, 17, 22, 26, 21, 23, 24, 20, 18, 17, 23, 23, 19,
       16, 22, 23, 26, 24, 45, 33, 39, 41, 38, 36], dtype=int64)